In [1]:
from fastai_old.text import *
import html
import spacy 

spacy.load('en')

In [2]:
num_verses = 3
verses = [0] * num_verses
chorus = ''

In [3]:
PATH=Path('data/')
LM_PATH=Path('data/model_lm/')
LM_PATH.mkdir(exist_ok=True)

In [4]:
# Uncomment these two lines for the smaller hip hop model
#itosv = pickle.load(open(LM_PATH/'tmp'/'itos-large-verse.pkl', 'rb'))
#itosc = pickle.load(open(LM_PATH/'tmp'/'itos-large-chorus.pkl', 'rb'))
# Uncomment these two lines for the smaller country model
itosv = pickle.load(open(LM_PATH/'tmp'/'itos-verse-country.pkl', 'rb'))
itosc = pickle.load(open(LM_PATH/'tmp'/'itos-chorus-country.pkl', 'rb'))
# Uncomment these two lines for the larger hip hop model
#itosv = pickle.load(open(LM_PATH/'tmp'/'itos-large-verse.pkl', 'rb'))
#itosc = pickle.load(open(LM_PATH/'tmp'/'itos-large-chorus.pkl', 'rb'))


stoiv = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itosv)})
stoic = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itosc)})

vsv=len(itosv)
vsc=len(itosc)
vsv, vsc

(5210, 2215)

In [5]:
trn_lm = np.empty([10,2])
val_lm = np.empty([10,2])

In [6]:
em_sz,nh,nl = 400,1150,3
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7
wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
mdv = LanguageModelData(PATH, 1, vsv, trn_dl, val_dl, bs=bs, bptt=bptt)
mdc = LanguageModelData(PATH, 1, vsc, trn_dl, val_dl, bs=bs, bptt=bptt)

DATA: 0 70
DATA: 0 70


In [7]:
learnerv= mdv.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learnerc= mdc.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

#learner.model.load_state_dict(wgts)

INIT <fastai_old.text.LanguageModelData object at 0x7f7201c28ac8>
INIT <fastai_old.text.LanguageModelData object at 0x7f7201c28b38>


In [8]:
def generate_text_verse(m, s, itos, stoi, l=20):
    m[0].bs=1  # Set batch size to 1
    m.eval()  # Turn off dropout
    m.reset()  # Reset hidden state
    m[0].bs=bs  # Put the batch size back to what it was

    ss = s.lower().split()
    si = [stoi[w] for w in ss]
    t = torch.autograd.Variable(torch.cuda.LongTensor(np.array([si])))
    
    res,*_ = m(t)
    
    output = s + ' '
    count = 0
    while True:
        #n = res[-1].topk(5)[1]  # top word
        n = torch.multinomial(res[-1].exp(), 10)  # drawing from probability distribution
        n = n[1] if n.data[0]==0 else n[0]
        if itos[int(n)] == '\n' and count > l:
            output += "\n"
            break
        if not any (x in itos[int(n)] for x in ['xbos', 'xfld']):
            output += itos[int(n)] + ' '
        res,*_ = m(n.unsqueeze(0).unsqueeze(0))  # sometimes need an extra .unsqueeze(0)
        count += 1
    return output

In [9]:
#Uncomment these two lines for the smaller hip hop model
learnerv.load('lm_30epochs-large-verse-set')
learnerc.load('lm_30epochs-large-chorus')
#Uncomment these two lines for the smaller country model
#learnerv.load('lm_30epochs-verse-country')
#learnerc.load('lm_30epochs-chorus-country')
#Uncomment these two lines for the larger hip hop model
#learnerv.load('lm_30epochs-verse-rap-verylarge')
#learnerc.load('lm_30epochs-chorus-rap-verylarge')

#These strings are the seeds for the verses
verse_strings = ["this is the way I", "somedays the paint", "which way does the road"]

#This is the seed string for the chorus
chorus_string = "these days they ca n't"

mv=learnerv.model
for i in range(num_verses):
    verses[i] = generate_text_verse(mv, verse_strings[i], itosv, stoiv, l=150)

mc=learnerc.model
chorus = generate_text_verse(mc, chorus_string, itosc, stoic, l=100)

RuntimeError: Error(s) in loading state_dict for SequentialRNN:
	size mismatch for 0.encoder.weight: copying a param with shape torch.Size([17190, 400]) from checkpoint, the shape in current model is torch.Size([5210, 400]).
	size mismatch for 0.encoder_with_dropout.embed.weight: copying a param with shape torch.Size([17190, 400]) from checkpoint, the shape in current model is torch.Size([5210, 400]).
	size mismatch for 1.decoder.weight: copying a param with shape torch.Size([17190, 400]) from checkpoint, the shape in current model is torch.Size([5210, 400]).

In [ ]:
def clean_output(x):
    return x.replace(" n't", "n't").replace(" ' til", "@'til").replace(" ' cause", "@'cause").replace(
        " '", "'").replace("@'til", " 'til").replace("@'cause", " 'cause").replace(" ,", ",")

In [ ]:
print('[Intro]')
for i in range(num_verses):
    print('\n[Verse ' + str(i + 1) + ']')
    print(clean_output(verses[i]))
    print('\n[Chorus]')
    print(clean_output(chorus))